# Bayes ingenuo

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

In [21]:
#
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [3]:
usePackage('R.utils')
usePackage('tm')

## Descarga los datos 

In [4]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [5]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

[1] TRUE

In [6]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [7]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [8]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt")
  lineas <- readLines(fd, warn=FALSE)
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n"))
}

In [9]:
# Creamos variables con los directorios donde se encuentran los datos
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### Spam

In [10]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
archivos_correos_spam_training <- archivos_correos_spam[1:1000]
archivos_correos_spam_testing <- archivos_correos_spam[1001:length(archivos_correos_spam)]

todo_spam <- sapply(archivos_correos_spam_training,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [11]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:1000]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[1001:length(archivos_correos_easy_ham)]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

## Preparación de corpus y bolsa de palabras

In [12]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 2)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [13]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [14]:
head(df_spam)

,terminos,frecuencia,densidad,ocurrencias
,<chr>,<dbl>,<dbl>,<dbl>
1,advantage,46,1.621248e-04,0.036
2,allowed,5,1.762226e-05,0.005
3,animal,5,1.762226e-05,0.004
4,arial,1111,3.915667e-03,0.133
5,attract,9,3.172007e-05,0.005
6,attracta,1,3.524453e-06,0.001


### Easy ham

In [15]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

In [22]:
head(df_easy_ham)

,terminos,frecuencia,densidad,ocurrencias
,<chr>,<dbl>,<dbl>,<dbl>
1,anything,73,5.327767e-04,0.065
2,anyway,59,4.306004e-04,0.053
3,around,135,9.852720e-04,0.105
4,aug,248,1.809981e-03,0.196
5,autodetect,3,2.189493e-05,0.002
6,boot,94,6.860413e-04,0.043


## Cálculo de probabilidad a posteriori

In [16]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c))
  }
  else
  {
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

## Clasificación

In [17]:
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  
  return (ifelse(hard_ham_spam_prueba > hard_ham_ham_prueba,
                        TRUE,
                        FALSE))
}

### Hard ham

In [18]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificlmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")]

In [19]:
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham_testing)
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing)

## Resultados

In [20]:
summary(easy_ham_res)
summary(spam_res)
summary(hard_ham_res)

   Mode   FALSE    TRUE 
logical     399       1 

   Mode   FALSE    TRUE 
logical      19     378 

   Mode   FALSE    TRUE 
logical     106     144 